# Reservoir Computing Project #

## ESN Network Setup ###

### Imports ###

In [38]:
from reservoirpy.datasets import lorenz
X = lorenz(n_timesteps=2000)

print(X)

[[ 1.          1.          1.        ]
 [ 1.1073228   1.79872164  0.96516196]
 [ 1.40967473  2.74000007  0.97272208]
 ...
 [-0.27309074 -0.50415907  8.98103121]
 [-0.35422832 -0.66790582  8.29546755]
 [-0.46425047 -0.88919121  7.66653225]]


In [39]:
from reservoirpy.nodes import Reservoir, Ridge

reservoir = Reservoir(units=100, lr=0.3, sr=1.25)
readout = Ridge(output_dim=3, ridge=1e-5)

In [40]:
esn = reservoir >> readout

In [41]:
predictions = esn.fit(X[:500], X[1:501]).run(X[501:-1])

Running Model-5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Model-5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Model-5: 500it [00:00, 5469.52it/s]           


Running Model-5: 100%|██████████| 1/1 [00:00<00:00,  6.83it/s]

Running Model-5: 100%|██████████| 1/1 [00:00<00:00,  6.80it/s]

...

Running Model-5:   0%|          | 0/1 [00:00<?, ?it/s]

Running Model-5: 374it [00:00, 3738.70it/s]           

Running Model-5: 759it [00:00, 3803.92it/s]

Running Model-5: 1140it [00:00, 3732.12it/s]

Running Model-5: 1498it [00:00, 3695.47it/s]

In [42]:
from reservoirpy.observables import rmse, rsquare

print("RMSE:", rmse(X[502:], predictions), "R^2 score:", rsquare(X[502:], predictions))

RMSE: 40.01341808980783 R^2 score: -7.363789313129299
